In [3]:
from web3 import Web3, Account
from eth_defi.aave_v3.deployment import fetch_deployment
from eth_defi.aave_v3.loan import supply, borrow
from eth_defi.aave_v3.constants import AAVE_V3_DEPLOYMENTS
from dotenv import load_dotenv
import os

load_dotenv()

# Setup Infura URL
infura_token = os.getenv("INFURA_TOKEN")
infura_url = f"https://mainnet.infura.io/v3/{infura_token}"
web3 = Web3(Web3.HTTPProvider(infura_url))

# Fetch Aave v3 deployment details
aave_v3 = fetch_deployment(
    web3,
    pool_address=AAVE_V3_DEPLOYMENTS["ethereum"]["pool"],
    data_provider_address=AAVE_V3_DEPLOYMENTS["ethereum"]["data_provider"],
    oracle_address=AAVE_V3_DEPLOYMENTS["ethereum"]["oracle"],
)

# Token addresses (WBTC, USDC)
WBTC_ADDRESS = "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599"  # WBTC contract address
USDC_ADDRESS = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eb48"  # USDC contract address

# Load ABIs for WBTC and USDC
# WBTC_ABI = [...]  # Replace with the actual ABI for WBTC
# USDC_ABI = [...]  # Replace with the actual ABI for USDC

# Create contract objects for tokens
# wbtc_contract = web3.eth.contract(address=WBTC_ADDRESS, abi=WBTC_ABI)
# usdc_contract = web3.eth.contract(address=USDC_ADDRESS, abi=USDC_ABI)


# Convert token amount to smallest unit
def to_token_unit(amount, decimals):
    return int(amount * 10**decimals)


# Create wallet function
def create_wallet():
    account = Account.create()
    wallet_address = account.address
    private_key = account.key.hex()
    print(f"New wallet created!")
    print(f"Address: {wallet_address}")
    print(f"Private Key: {private_key}")
    return wallet_address, private_key


# Deposit WBTC into Aave
def deposit_wbtc(wallet_address, private_key, amount):
    amount_in_wei = to_token_unit(amount, 8)
    approve_fn, supply_fn = supply(
        aave_v3_deployment=aave_v3,
        token=WBTC_ADDRESS,
        amount=amount_in_wei,
        wallet_address=wallet_address,
    )

    tx = approve_fn.build_transaction({"from": wallet_address, "gas": 2000000})
    signed = web3.eth.account.sign_transaction(tx, private_key)
    tx_hash = web3.eth.send_raw_transaction(signed.rawTransaction)
    print(f"Approval transaction hash: {tx_hash.hex()}")

    tx = supply_fn.build_transaction({"from": wallet_address, "gas": 2000000})
    signed = web3.eth.account.sign_transaction(tx, private_key)
    tx_hash = web3.eth.send_raw_transaction(signed.rawTransaction)
    print(f"Deposit transaction hash: {tx_hash.hex()}")
    return tx_hash

def deposit_2(wallet_address, private_key, amount):
    amount_in_wei = to_token_unit(amount, 8)
    borrow_fn = borrow(
        aave_v3_deployment=aave_v3,
        token=WBTC_ADDRESS,
        amount=amount_in_wei,
        wallet_address=wallet_address,
    )
    
    tx = borrow_fn.build_transaction({"from": wallet_address})
    signed = wallet_address.sign_transaction_with_new_nonce(tx)
    tx_hash = web3.eth.send_raw_transaction(signed.rawTransaction)
    # assert_transaction_success_with_explanation(web3, tx_hash)
    
    print(tx)
    print(signed)
    print(tx_hash)
    
    print("====================================")

# Borrow USDC from Aave
def borrow_usdc(wallet_address, private_key, amount):
    amount_in_mwei = to_token_unit(amount, 6)
    borrow_fn = borrow(
        aave_v3_deployment=aave_v3,
        token=USDC_ADDRESS,
        amount=amount_in_mwei,
        wallet_address=wallet_address,
    )

    tx = borrow_fn.build_transaction({"from": wallet_address, "gas": 2000000})
    signed = web3.eth.account.sign_transaction(tx, private_key)
    tx_hash = web3.eth.send_raw_transaction(signed.rawTransaction)
    print(f"Borrow transaction hash: {tx_hash.hex()}")
    return tx_hash


# Test deposit, borrow, and withdraw functions
def test_deposit_and_borrow():
    wallet_address, private_key = create_wallet()
    print(
        f"IMPORTANT: Ensure the wallet ({wallet_address}) is funded with ETH for gas fees."
    )
    
    deposit_2(wallet_address, private_key, 0.1)

    # try:
    # except Exception as e:
    # print(f"Error during deposit: {e}")

    try:
        print("Testing deposit of 0.1 WBTC...")
        deposit_tx = deposit_wbtc(wallet_address, private_key, 0.1)
        print(f"Deposit transaction successful: {deposit_tx.hex()}")
    except Exception as e:
        print(f"Error during deposit: {e}")

    try:
        print("Testing borrowing of 50 USDC...")
        borrow_tx = borrow_usdc(wallet_address, private_key, 50)
        print(f"Borrow transaction successful: {borrow_tx.hex()}")
    except Exception as e:
        print(f"Error during borrowing: {e}")


if __name__ == "__main__":
    test_deposit_and_borrow()

New wallet created!
Address: 0x45a7394BB610c2e86FE4773366caC925Fb8a0A9C
Private Key: 0xb6f19fd1dfc2623e5a01b2a8bdf324953acabbd2a2b424880772d8608916c222
IMPORTANT: Ensure the wallet (0x45a7394BB610c2e86FE4773366caC925Fb8a0A9C) is funded with ETH for gas fees.


AttributeError: 'str' object has no attribute 'address'

In [8]:
from web3 import Web3, Account
from eth_defi.aave_v3.deployment import fetch_deployment
from eth_defi.aave_v3.loan import supply, borrow
from eth_defi.aave_v3.constants import AAVE_V3_DEPLOYMENTS
from dotenv import load_dotenv
import os

load_dotenv()

infura_token = os.getenv("INFURA_TOKEN")
infura_url = f"https://mainnet.infura.io/v3/{infura_token}"
web3 = Web3(Web3.HTTPProvider(infura_url))

from web3 import Web3

WBTC_ADDRESS = "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599"

ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function",
    },
    {
        "constant": False,
        "inputs": [
            {"name": "_spender", "type": "address"},
            {"name": "_value", "type": "uint256"},
        ],
        "name": "approve",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function",
    },
]

wbtc_contract = web3.eth.contract(
    address=Web3.to_checksum_address(WBTC_ADDRESS), abi=ERC20_ABI
)

aave_v3 = fetch_deployment(
    web3,
    pool_address=AAVE_V3_DEPLOYMENTS["ethereum"]["pool"],
    data_provider_address=AAVE_V3_DEPLOYMENTS["ethereum"]["data_provider"],
    oracle_address=AAVE_V3_DEPLOYMENTS["ethereum"]["oracle"],
)

def to_token_unit(amount, decimals):
    return int(amount * 10**decimals)


def create_wallet():
    account = Account.create()
    wallet_address = account.address
    private_key = account.key.hex()
    print(f"New wallet created!")
    print(f"Address: {wallet_address}")
    print(f"Private Key: {private_key}")
    return wallet_address, private_key


def deposit_2(wallet_address, private_key, amount):
    try:
        print("====================================")
        print(wallet_address, private_key, amount)
        amount_in_wei = to_token_unit(amount, 8)

        borrow_fn = borrow(
            aave_v3_deployment=aave_v3,
            token=wbtc_contract,
            amount=amount_in_wei,
            wallet_address=wallet_address,
        )

        print("====================================")
        print(borrow_fn)
        print("====================================")

        tx = borrow_fn.build_transaction({"from": wallet_address})
        nonce = web3.eth.get_transaction_count(wallet_address)
        tx["nonce"] = nonce
        signed_tx = web3.eth.account.sign_transaction(tx, private_key)

        tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
        print(f"Transaction Hash: {tx_hash.hex()}")
        print("====================================")
    except Exception as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    wallet_address, private_key = create_wallet()

    deposit_2(wallet_address, private_key, 0.1)

New wallet created!
Address: 0x398F0b205290Dd1E5e6e6bF1586b613bf1C01C03
Private Key: 0xbd4c8d3c27a3cf9e979436d1a8a2fc2ff0d49acf311e3301ca4483b128710925
0x398F0b205290Dd1E5e6e6bF1586b613bf1C01C03 0xbd4c8d3c27a3cf9e979436d1a8a2fc2ff0d49acf311e3301ca4483b128710925 0.1
<Function borrow(address,uint256,uint256,uint16,address) bound to ('0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', 10000000, <AaveV3InterestRateMode.VARIABLE: 2>, 0, '0x398F0b205290Dd1E5e6e6bF1586b613bf1C01C03')>
Error: execution reverted: 34
